# Parte 1: Ciclo de vida de un modelo - Registro / Almacenamiento del Modelo con Unity Catalog 
*   **Autor:** Carolina Torres Zapata
*   **Fecha:** 2025-11-24
*   **Contexto:**
Este notebook implementa el **escenario ideal** de operación productiva solicitado en la prueba (Punto 4.a). Utiliza **Unity Catalog** como *Model Registry* centralizado.

**Ventajas de este enfoque:**
1.  **Nombre Estable:** El modelo se invoca por un nombre semántico (`dev.ml_models.churn_model`) en lugar de un hash aleatorio.
2.  **Gobierno de Datos:** El modelo reside en el mismo catálogo que los datos, permitiendo control de accesos unificado.
3.  **Gestión de Versiones:** Permite cargar versiones específicas (ej. `Version 1`) o Alias (ej. `@Champion`, `@Production`).

## 1. Importar Librerías

In [0]:
import mlflow
import mlflow.sklearn
import pandas as pd
from pyspark.sql.functions import col

## 2. Configuración del Entorno de Gobierno
Aseguramos que exista el esquema (base de datos) dentro del Unity Catalog donde residirán nuestros modelos registrados.
*   **Catalog:** `dev`
*   **Schema:** `ml_models` (espacio dedicado para artefactos de ML)

In [0]:
%sql
CREATE SCHEMA IF NOT EXISTS dev.ml_models;

## 3. Definición del URI del Modelo Gobernado
 
En lugar de buscar dinámicamente el "mejor run" (como en el método de fallback), aquí ingresamos los valores que el equipo de Data Science ha registrado la versión estable en Unity Catalog.

Utilizamos la estructura de tres niveles de UC: `catalogo.esquema.nombre_modelo`.

In [0]:
# URI del modelo registrado en Unity Catalog
model_name = "dev.ml_models.churn_model"
model_version = 1

model_uri_uc = f"models:/{model_name}/{model_version}"

print("📌 Model URI:", model_uri_uc)


📌 Model URI: models:/dev.ml_models.churn_model/1


## 4. Carga del Modelo para Inferencia
Usamos el sabor `pyfunc` de MLflow. Esto es una buena práctica operativa porque abstrae la librería subyacente (Scikit-Learn).

In [0]:
loaded_model_uc = mlflow.pyfunc.load_model(model_uri_uc)
print("✔ Modelo UC cargado exitosamente como PyFunc MLflow")

✔ Modelo UC cargado exitosamente como PyFunc MLflow


### 4.1. Leer tabla Silver (Simulando datos nuevos)
**Simulación de Pipeline de Inferencia (Batch)**
Leemos los datos más recientes de la capa **Silver**.
*   **Nota:** Al igual que en el entrenamiento, separamos estrictamente los identificadores (`customerID`) de las variables predictivas (`X_new`) para evitar sesgos o errores de esquema.

In [0]:
# 1. Leer tabla Silver (Simulando datos nuevos)
table_name = "dev.silver.churn_data"
df_spark = spark.read.table(table_name)

# Tomamos una muestra de 10 clientes para la demo
df_inference = df_spark.sample(fraction=0.1, seed=42).limit(10).toPandas()

# 2. Separar Metadatos (Lo que el modelo NO debe ver)
# Guardamos ID y Label real en un dataframe aparte para el reporte final
cols_meta = ["customerID", "Churn"]
df_meta = df_inference[cols_meta].copy()

# 3. Crear X para el modelo (Solo las features procesadas)
# Borramos las columnas que no son features
X_new = df_inference.drop(columns=cols_meta)

print("--- Datos de Entrada al Modelo (X_new) ---")
#print(f"Columnas: {X_new.columns.tolist()}")
display(X_new.head(2))

INFO:py4j.clientserver:Received command c on object id p0


--- Datos de Entrada al Modelo (X_new) ---


gender_Male,SeniorCitizen,Partner,Dependents,PhoneService,MultipleLines_No_phone_service,MultipleLines,InternetService_Fiber_optic,InternetService_No,OnlineSecurity_No_internet_service,OnlineSecurity,OnlineBackup_No_internet_service,OnlineBackup,DeviceProtection_No_internet_service,DeviceProtection,TechSupport_No_internet_service,TechSupport,StreamingTV_No_internet_service,StreamingTV,StreamingMovies_No_internet_service,StreamingMovies,Contract_One_year,Contract_Two_year,PaperlessBilling,PaymentMethod_Credit_card_automatic,PaymentMethod_Electronic_check,PaymentMethod_Mailed_check,tenure,MonthlyCharges,TotalCharges
0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.8400143297628127,1.5201556452330371,1.5684111969587773
0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.5322605130033284,1.6431286232453304,2.402200981291416


### 4.2. Ejecución de Inferencia
El modelo registrado en Unity Catalog recibe el DataFrame de Pandas (o Spark) y devuelve las predicciones.

In [0]:
preds = loaded_model_uc.predict(X_new)
print("✅ Inferencia finalizada.")

✅ Inferencia finalizada.


## 5. Generación de Reporte de Negocio
Convertimos las predicciones técnicas (arrays de 0s y 1s) en información útil para el negocio, re-asociando los resultados con los `customerID` originales.

In [0]:
# Cruzamos las predicciones con el `customerID` original para que el reporte sea accionable.

# 1. Unimos todo en un reporte final
reporte = df_meta.copy() # Empezamos con ID y Valor Real

# 2. Agregamos las predicciones
#reporte["Probabilidad_Fuga"] = probs.round(4)
reporte["Prediccion_Modelo"] = preds

# 4. Visualización Final
print("--- REPORTE FINAL PARA SOPORTE A OPERACIÓN ---")
#cols_mostrar = ["customerID", "Probabilidad_Fuga", "Prediccion_Modelo", "Alerta_Gestion", "Churn"]
cols_mostrar = ["customerID",  "Prediccion_Modelo", "Churn"]
display(reporte[cols_mostrar])

--- REPORTE FINAL PARA SOPORTE A OPERACIÓN ---


customerID,Prediccion_Modelo,Churn
8008-ESFLK,0,0
1555-DJEQW,0,1
0519-XUZJU,1,1
4750-UKWJK,0,0
2516-XSJKX,0,0
0356-ERHVT,0,0
3055-OYMSE,0,0
6096-EGVTU,0,0
8319-QBEHW,0,0
7110-BDTWG,0,0
